Predictions

Imports

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
import autoreload
%load_ext autoreload
%autoreload 2
import python_code.Scripts as sc
import python_code.Reference as ref

In [2]:
model= pickle.load(open('final_model.sav','rb'))

2018 data

In [3]:
df= pd.read_csv('./data/qc_pub_fy2018.csv')
df.head()

,FSAFIL1,FSAFIL2,FSAFIL3,FSAFIL4,FSAFIL5,FSAFIL6,FSAFIL7,FSAFIL8,FSAFIL9,FSAFIL10,...,WORK12,WORK13,WORK14,WORK15,WORK16,FSNONCIT,FSDIS,FSELDER,FSKID,STATENAME
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,1,0,Connecticut
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,1,0,0,Connecticut
2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,1,0,0,Connecticut
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,Connecticut
4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,1,0,0,Connecticut


In [4]:
df.shape

(43738, 814)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43738 entries, 0 to 43737
Columns: 814 entries, FSAFIL1 to STATENAME
dtypes: float64(373), int64(440), object(1)
memory usage: 271.6+ MB


Predictions

In [6]:
def clean_data_to_predict(dataset):
    """Cleans new datasets for processing through prediciton"""
    fullname_df= pd.DataFrame()
    fullname_df['state']=dataset['STATENAME']
    for feature in ref.corr_features['Table 1']:
        try:
            fullname_df[feature]= dataset[feature].astype('float64')
        except:
            continue
    fullname_df['VEHICLEA']=fullname_df['VEHICLEA'].fillna(1) #set nulls to 1 for "no car"
    fullname_df=fullname_df.dropna() #drop the remaining null values out of the reduced column set
    return fullname_df
    

In [7]:
df= sc.clean_data_to_predict(df)

In [8]:
ss= StandardScaler()
X= ss.fit_transform(df.drop(columns=['state']))
df['predict']=model.predict(X)
df

,state,FSERNDED,TPOV,FSNONCIT,FSUSIZE,RAWNET,REALPROP,FSDIS,FSTANF,FSSTDDE2,...,FSEARN,FSERNDE2,FSWAGES,FSSSI,SHELDED,FSGA,FSSLTDED,RAWERND,FSASSET,predict
0,Connecticut,0.0,92.0,0.0,1.0,176.0,0.0,0.0,0.0,240.0,...,0.0,0.0,0.0,0.0,584.0,0.0,584.0,0.0,0.0,0.0
1,Connecticut,42.0,140.0,0.0,1.0,239.0,0.0,1.0,0.0,240.0,...,212.0,63.0,212.0,0.0,968.0,0.0,969.0,42.0,0.0,0.0
2,Connecticut,0.0,127.0,0.0,1.0,149.0,0.0,1.0,0.0,240.0,...,0.0,0.0,0.0,0.0,969.0,0.0,969.0,0.0,0.0,0.0
3,Connecticut,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Connecticut,0.0,111.0,0.0,1.0,151.0,0.0,1.0,0.0,240.0,...,0.0,0.0,0.0,735.0,802.0,0.0,802.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43733,Guam,0.0,11.0,0.0,2.0,0.0,0.0,0.0,0.0,150.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
43734,Guam,215.0,79.0,0.0,2.0,468.0,0.0,0.0,0.0,393.0,...,1076.0,287.0,1076.0,0.0,342.0,0.0,72.0,215.0,0.0,1.0
43735,Guam,206.0,61.0,0.0,3.0,504.0,0.0,0.0,0.0,321.0,...,1031.0,206.0,1031.0,0.0,124.0,0.0,0.0,206.0,0.0,1.0
43736,Guam,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
df.to_csv('./data/2018_indicators/preds_all.csv',index=None)

In [10]:
pred_df= pd.DataFrame(df.groupby('state')['predict'].value_counts().unstack()).rename(columns={0.0:'No',1.0:'Yes'})

In [11]:
pred_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53 entries, Alabama to Wyoming
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   No      53 non-null     int64
 1   Yes     53 non-null     int64
dtypes: int64(2)
memory usage: 1.2+ KB


In [14]:
pred_df.to_csv('./data/2018_indicators/preds_target.csv',index='state')

GIS Analysis files

In [15]:
coef_df= pd.read_csv('./data/2018_indicators/coef.csv')
coef_df['diff']= (coef_df['pred_0']-coef_df['pred_1']).abs()
coef_diff= coef_df.drop(columns=['pred_0','pred_1'])

In [17]:
coef_diff= coef_diff.sort_values(by=['diff'],ascending=False).reset_index(drop=True)
top= coef_diff.head(15)
top.to_csv('./data/2018_indicators/top_impacts.csv',index=None)

In [18]:
coef_2018= sc.top_coef_df(top,df).set_index('state')
coef_2018.astype('int64').to_csv('./data/2018_indicators/coef_2018_all.csv',index='state')

In [19]:
coef_2018.groupby('state').mean().astype('int64').to_csv('./data/2018_indicators/coef_2018_state_avg.csv',index='state')